# Bayesian Deep Learning: MC dropout

## 1. (Conventional) dropout

* ReLU는 약간 인지편향적인 감이 있고, GeLU 등으로 대체되는 추세임
* Dropout은 완전 무작위적으로 connection을 제거하니까...

`-` Fully Connected Network

* 모든 뉴런이 연결되어 있음
* 순차적 학습

`-` Dropout

* DNN의 업데이트 횟수는 매우 많음 $\to$ 업데이트 과정에서 학습의 방향을 바꾸자.
* DNN은 일반적으로 확률적 요소가 포함되어 있기 때문에 학습이 안정적. 수렴함.
* Dropout을 사용하면 step마다 완전 무작위로 뉴런의 연결을 끊어냄.
> Masking. 두 뉴런을 연결하지 않음.

`-` Stochastic masking

* 마스킹 확률: $1 - p$
* 일종의 앙상?블 관점으로 얘기할 수 있었는데, 일반적인 앙상블이라고 말하긴 힘들어.

1. Activated parameter들이 계속 바뀜(weights and biases). 마스킹된 파라미터는 스텝마다 다름
2. 추론 시, 업데이트된 파라미터의 뒷부분 스텝들의 평균을 사용한다. 마지막 업데이트가 전체 파라미터를 대변하지 않는다.
3. 

> 최종적으로 각 파라미터들은 업데이트된 횟수가 다름. 이 경우 평가 시에 가중치는 훈련 뒷부분 몇 개의 step에서 평균적인 값으로 적용함.

`-` Test procedure

* 최종 평가 단계에서 마스킹되지 않을 확률을 곱해준다.
* Conventional ensemble이 아니다: 각 모형이 독립적이지 않음
* Boosting도 아니다: 마스킹은 완전 확률적으로 수행. 이전 스텝에서의 정보를 반영하지 않음. $\to$ DNN은 앙상블할 필요가 없이 Dropout만으로 충분하다.
* 훈련 스텝마다 다른 네트워크를 사용한다.

`-` Dropout의 특징

* dropout을 사용하면 total step만큼 파라미터가 갱신되지 않는다. 따라서 에폭을 증가시켜줄 필요가 있다.
* 그래디언트의 흐름을 더 부드럽게 만들어준다. vanishing/exploding을 막아준다.

## 2. Bayesian DL, MC dropout

* 기존 Dropout 코드를 거의 그대로 가져다 사용할 수 있음.
* DNN의 필연적인 Randomness를 수학적으로 해석 $\to$ 수렴 상태인 모델을 더 훈련하여 다양한 $\hat y$을 산출하고 이를 분포로 나타내고자 했음 (해당 사실이 완전히 규명되지는 않음)
* Gradient가 0이 나와도 업데이트가 수행됨: Perturbation으로 Randomness를 더 확보...

`-` Dropout을 Bayesian Posterior로 해석

* Prior and Variational Bayes

1. Prior and Model

$$\begin{align}
& \text{Prior:}~p(\boldsymbol w) ~ \text{where} ~ \boldsymbol w = (\bf W_1, \cdots, \bf W_L) \\
& \text{Model:}~ \hat y (x, \boldsymbol w)
\end{align}$$

> $\bf W_i$는 각 레이어의 파라미터들로 이뤄진 매트릭스이며, $p(\boldsymbol w)$는 $\bf W_i$의 Prior이다.

2. $q(\boldsymbol w)$ in VB: $\bf W_i \sim q$

$$W_i = M_i ⊙ \{z_{ij}\}_{(i, j) \in [K_i]\times [K_{i-1}]} ≡ M_i⊙Z_j$$

> $z_{ij} \sim \text{Bernoulii}(p_i)$. $p_i$는 dropout의 확률로 설정. 상수. Discrete Prior, Continuous Posterior
>
> $M_i$에 대해서 ELBO를 최적화. 해당 값에 따라 $q$가 바뀜

3. negative ELBO

4. MC integration of the first term

> dropout을 사용하여 $M$을 얻어낸다.

`-` Bayesian Prediction

1. Formula

$$p(y*~|~x*) = \int p(y*~|~x*, \boldsymbol w)q(\boldsymbol w) d\boldsymbol w$$

2. Prediction by sampling of $\{(W_1^{(t)}, \cdots, W_L^{(t)}))\}_{t=1}^T$

$$\mathbb E_q(y*~|~x*)[y*] \approx \frac{1}{T} \sum_{t=1}^T$$

> $\hat M z$를 얻은 상태에서, random $z$를 이용하여 계속 샘플을 생성.
>
> 끝쪽에서 훈련만 더 시켜서 mean, quantile, sampling하기만 하면 됨.
>
> 최종 예측은 그 결과들을 평균내든 해서 산출할 수 있음.

`-` MC dropout과 Ensemble Method의 차이

* MC dropout은 각 모델이 하나의 step일 뿐임
* Ensemble Method는 모델을 계속 만들어야 함
* Credential Interval은 제대로 산출되지 않음

## 3. MC dropout example

* VAE 계통에서는 가장 좋은 성능 (기존 dropout보다 좋음)

`-` sudo code

```Python
model.train()   ## 이미 학습된 모델을 다시 학습
dummy_input = torch.randn(1, 1, 28, 28)
num_mc_samples = 100

predictions = []

for _ in range(num_mc_samples):
    with torch.no_grad():
        output = model(dummy_input)
        predictions.append(output)

predictions_tensor = torch.statck(predictions, dim = 1)
mean_prediction = torch.mean(predictions_tensor, dim = 1)
variance_prediction = torch.var(predictions_tensor, dim = 1)
```

> MC dropout을 하고자 한다면, 구간을 잘 추정하지 못한다는 단점을 잘 해소하는 것이 중요함

`-` Mutual Information

* measure가 얼마나 불확실한가? $p(x, y), p(x)p(y)$에서 두 분포 간 KL divergence

$$H(y^*~|~x^*) - \mathbb E_q [H(y^*~|~x^*, \boldsymbol w*)]$$

> $x, y$가 독립이냐 아니냐로 유도됨. mutual information이 크면, 관련성이 많으므로 도움이 많이 됨.

`-` Providing the confidence

* 예측에 대한 uncertainty가 존재함 $\to$ 예측만 잘하는 것이 아닌 uncertainty 통제를 위해서는 MC dropout이 유용할 수 있다...?
* 추정을 위한 uncertainty와 (확률값 산출. credential interval), 가지는 값에 의해 구조적으로 생기는 uncertainty가 (상대적 값. 0.2보다 0.4가 0 추정에 uncertainty가 높음) 존재.

## Mutual Information

* Uncertancy를 측정하는 지표 중 하나 (Confidence Interval / Credential Interval)
* Classification의 예측은 대부분 확률을 산출한다. 이 경우 확률값 자체가 정확한 값이 아니기 때문에 Confidence Interval이 존재한다.

`-` Entropy 계산은 기본적으로 확률을 대상으로 한다.

$$\begin{align}
\text{Entropy} & = -\sum_k p_k \log p_k \\
I & = - \sum_k p(y=k~|~x, \eta) \log p(y=k~|~x, \eta) \\
\text{Data} & = - E_{\theta} \left[ - \sum_k p(y=k~|~x, \theta) \log p(y=k~|~x, \theta) \right]
\end{align}$$

* 적분을 먼저한 것
* 확률 계산 하고 적분
> Posterior의 분산이 작을수록 근사값이 실제와 유사해짐
>
> Entropy를 적분하고 확률값 계산 / Entropy의 확률값을 계산하고 적분 $\to$ 

* $y, \theta ~|~ \eta$간의 Mutual Information. 완전히 동일한 posterior에 대해서도 $x$가 어떤 값을 가지는지에 따라 해당 값이 달라짐.
* mutual information이 높다는 것은 posterior에 의한 entropy의 변동이 심하다. 조금 더 uncertant하다. 특정 데이터의 mutual information이 더 높아서 신뢰도를 의심해야 될 수 있다.

`-` 독립과 KL-divergence

$$\begin{align}
p(x, y) & = p(x)p(y) \\
\frac{p(x, y)}{p(x), p(y)} & = 1 \\
\Rightarrow \text{KL}(p(x, y)~||~ p(x)p(y)) & = \int \log \left(\frac{p(x, y)}{p(x)p(y)} \right) p(x, y) dx dy
\end{align}$$

* 예측 시 독립인 관계는 필요 없음. $x$로 $y$를 알아내려는 건데, 독립이면? $y~|~x$는 아무 의미도 없음
* mutual information이 클 수록 정보를 많이 공유하고, 종속적. $y$의 행동을 예측할 때 $x$가 유용함

`-` 조건부 독립과 mutual information

* $p(y, \theta~|~\eta) = p(y~|~\eta)p(\theta~|~\eta)$의 관점. 조건부 독립 관점에서의 mutual information

$$\begin{align}
p(y, \theta~|~\eta) & = p(y~|~\eta)p(\theta~|~\eta) \\
\Rightarrow & \int \log \frac{p(y, \theta~|~\eta)}{p(y~|~\eta)p(\theta~|~\eta)} p(y, \theta~|~\eta) dy d\theta \\
& = \int \log \frac{p(y~|~ \theta, \eta)p(\theta~|~\eta)}{p(y~|~\eta)p(\theta~|~\eta)} p(y, \theta~|~\eta) dy d\theta \\
& = \int \log p(y~|~\theta, \eta) p(y, \theta,~|~\eta) dy d\theta - \int \log p(y~|~\eta)p(y, \theta~|~\eta) dy d\theta \\
& = \int
\end{align}$$